In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn import model_selection, datasets, svm
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA

import data_process_funcs
import meta_dataframe_functions


In [10]:
big_df =  meta_dataframe_functions.blank_meta_df()
meta_dataframe_functions.load_meta_df(big_df,'Hardware')


In [11]:
big_df.head()

,nr_qubits,backend,sim,circuit_type,file_path
0,4,torino,False,1,../Hardware_results/4q/4q_ibm_torino1.csv
1,4,torino,False,2,../Hardware_results/4q/4q_ibm_torino2.csv
2,4,torino,False,3,../Hardware_results/4q/4q_ibm_torino3.csv
3,4,brisbane,False,1,../Hardware_results/4q/4q_ibm_brisbane1.csv
4,4,brisbane,False,2,../Hardware_results/4q/4q_ibm_brisbane2.csv


In [ ]:
big_df['circuit_type'] = pd.to_numeric(big_df['circuit_type'], downcast='integer', errors='coerce')
big_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12 entries, 0 to 11
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   nr_qubits     12 non-null     object
 1   backend       12 non-null     object
 2   sim           12 non-null     bool  
 3   circuit_type  12 non-null     int8  
 4   file_path     12 non-null     object
dtypes: bool(1), int8(1), object(3)
memory usage: 408.0+ bytes


In [14]:
label_encoder = LabelEncoder()
big_df['backend'] = label_encoder.fit_transform(big_df['backend'])
big_df.head()

,nr_qubits,backend,sim,circuit_type,file_path
0,4,1,False,1,../Hardware_results/4q/4q_ibm_torino1.csv
1,4,1,False,2,../Hardware_results/4q/4q_ibm_torino2.csv
2,4,1,False,3,../Hardware_results/4q/4q_ibm_torino3.csv
3,4,0,False,1,../Hardware_results/4q/4q_ibm_brisbane1.csv
4,4,0,False,2,../Hardware_results/4q/4q_ibm_brisbane2.csv


In [16]:
df_arr = data_process_funcs.arr_dfs_of_qubit_sizes('Hardware')
df_4q_H = df_arr[0]
df_4q_H.head()

,circuit_type,backend,nr_qubits,totalError,0000,0001,0010,0011,0100,0101,0110,0111,1000,1001,1010,1011,1100,1101,1110,1111
0,1,torino,4,766,1.0,0.513055,0.268930,0.067885,0.049608,0.007833,0.007833,0.060052,0.015666,0.000000,0.000000,0.000000,0.001305,0.001305,0.003916,0.002611
1,1,torino,4,740,1.0,0.495946,0.255405,0.082432,0.059459,0.016216,0.018919,0.040541,0.016216,0.002703,0.002703,0.001351,0.002703,0.000000,0.002703,0.002703
2,1,torino,4,776,1.0,0.416237,0.337629,0.083763,0.065722,0.006443,0.010309,0.052835,0.012887,0.002577,0.001289,0.000000,0.001289,0.000000,0.001289,0.007732
3,1,torino,4,799,1.0,0.468085,0.255319,0.093867,0.067584,0.011264,0.015019,0.056320,0.015019,0.001252,0.003755,0.000000,0.002503,0.000000,0.002503,0.007509
4,1,torino,4,785,1.0,0.496815,0.271338,0.082803,0.053503,0.014013,0.008917,0.045860,0.016561,0.001274,0.003822,0.000000,0.001274,0.000000,0.001274,0.002548


In [19]:
def features_to_int(df):
    df_ = df
    label_encoder = LabelEncoder()
    df_['backend'] = label_encoder.fit_transform(df_['backend'])
    df_['circuit_type'] = pd.to_numeric(df_['circuit_type'], downcast='integer', errors='coerce')
    df_ = df_.drop('nr_qubits', axis = 1)
    return df_

In [ ]:
df_4q_H = features_to_int(df_4q_H)


In [21]:

df_4q_H.head()

,circuit_type,backend,totalError,0000,0001,0010,0011,0100,0101,0110,0111,1000,1001,1010,1011,1100,1101,1110,1111
0,1,1,766,1.0,0.513055,0.268930,0.067885,0.049608,0.007833,0.007833,0.060052,0.015666,0.000000,0.000000,0.000000,0.001305,0.001305,0.003916,0.002611
1,1,1,740,1.0,0.495946,0.255405,0.082432,0.059459,0.016216,0.018919,0.040541,0.016216,0.002703,0.002703,0.001351,0.002703,0.000000,0.002703,0.002703
2,1,1,776,1.0,0.416237,0.337629,0.083763,0.065722,0.006443,0.010309,0.052835,0.012887,0.002577,0.001289,0.000000,0.001289,0.000000,0.001289,0.007732
3,1,1,799,1.0,0.468085,0.255319,0.093867,0.067584,0.011264,0.015019,0.056320,0.015019,0.001252,0.003755,0.000000,0.002503,0.000000,0.002503,0.007509
4,1,1,785,1.0,0.496815,0.271338,0.082803,0.053503,0.014013,0.008917,0.045860,0.016561,0.001274,0.003822,0.000000,0.001274,0.000000,0.001274,0.002548


In [22]:
df_4q_H.tail()

,circuit_type,backend,totalError,0000,0001,0010,0011,0100,0101,0110,0111,1000,1001,1010,1011,1100,1101,1110,1111
337,3,0,933,1.0,0.516613,0.082529,0.035370,0.088960,0.060021,0.003215,0.002144,0.116827,0.067524,0.003215,0.005359,0.008574,0.008574,0.001072,0.0
338,3,0,964,1.0,0.502075,0.082988,0.038382,0.085062,0.044606,0.004149,0.005187,0.128631,0.090249,0.003112,0.004149,0.007261,0.003112,0.001037,0.0
339,3,0,993,1.0,0.496475,0.091641,0.040282,0.078550,0.049345,0.005035,0.005035,0.132931,0.078550,0.001007,0.002014,0.007049,0.011078,0.001007,0.0
340,3,0,871,1.0,0.493685,0.090700,0.029851,0.087256,0.066590,0.005741,0.004592,0.118255,0.084960,0.002296,0.001148,0.006889,0.006889,0.001148,0.0
341,3,0,908,1.0,0.511013,0.090308,0.034141,0.067181,0.064978,0.004405,0.003304,0.128855,0.083700,0.002203,0.004405,0.003304,0.002203,0.000000,0.0


In [23]:
Y = df_4q_H[['backend']]
X = df_4q_H.drop('backend',axis = 1)

X_train, X_test, Y_train, Y_test = model_selection.train_test_split(
    X,Y,train_size=40,shuffle = True,random_state=42)

In [24]:
X.head()

,circuit_type,totalError,0000,0001,0010,0011,0100,0101,0110,0111,1000,1001,1010,1011,1100,1101,1110,1111
0,1,766,1.0,0.513055,0.268930,0.067885,0.049608,0.007833,0.007833,0.060052,0.015666,0.000000,0.000000,0.000000,0.001305,0.001305,0.003916,0.002611
1,1,740,1.0,0.495946,0.255405,0.082432,0.059459,0.016216,0.018919,0.040541,0.016216,0.002703,0.002703,0.001351,0.002703,0.000000,0.002703,0.002703
2,1,776,1.0,0.416237,0.337629,0.083763,0.065722,0.006443,0.010309,0.052835,0.012887,0.002577,0.001289,0.000000,0.001289,0.000000,0.001289,0.007732
3,1,799,1.0,0.468085,0.255319,0.093867,0.067584,0.011264,0.015019,0.056320,0.015019,0.001252,0.003755,0.000000,0.002503,0.000000,0.002503,0.007509
4,1,785,1.0,0.496815,0.271338,0.082803,0.053503,0.014013,0.008917,0.045860,0.016561,0.001274,0.003822,0.000000,0.001274,0.000000,0.001274,0.002548


In [15]:
def fit_and_get_val_score_SVM(X_train,Y_train,X_val,Y_val,kernel_ = 'rbf',ravel = True):
    baseline = SVC(kernel=kernel_)
    if ravel:
        Y_train_1d = Y_train.to_numpy()
        Y_train_1d = Y_train_1d.ravel()
    else:
        Y_train_1d = Y_train
    baseline.fit(X_train, Y_train_1d)
    Cscore = baseline.score(X_val, Y_val)
    print("SVM Validation Accuracy:", Cscore)
    return baseline